In [1]:
import os, json, random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, SubsetRandomSampler, Dataset

import optuna
from metrics import metrics_from_preds
from preprocess import load_data

In [2]:
def set_seed(seed: int = 42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Para reproducibilidad estricta; si alguna op no soporta determinismo, solo avisa.
    torch.use_deterministic_algorithms(True, warn_only=True)
    torch.set_num_threads(1)

In [3]:
set_seed(42)

# Forzamos CPU para bit-a-bit. (Si luego quieres GPU, cambia a 'cuda' sabiendo que habrá
# pequeñas variaciones numéricas.)
device = torch.device("cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cpu


In [4]:
vids_training, labels_training, vids_val, labels_val, vids_test, labels_test = load_data()

In [5]:
class VideoDataset(Dataset):
    """Envuelve listas de (video, label).
       Espera 'video' con shape (T, F). Devuelve (x: Tensor[T, F], y: LongTensor[])."""
    def __init__(self, videos, labels):
        self.videos = videos
        self.labels = labels
    def __len__(self):
        return len(self.videos)
    def __getitem__(self, idx):
        x = torch.tensor(np.array(self.videos[idx]), dtype=torch.float32)  # (T, F)
        y = torch.tensor(self.labels[idx], dtype=torch.long)               # ()
        return x, y

In [6]:
def collate_pad(batch):
    """Collate simple asumiendo T fijo en el dataset (sin padding)."""
    xs, ys = zip(*batch)
    x = torch.stack(xs, dim=0)  # (B, T, F)
    y = torch.stack(ys, dim=0)  # (B,)
    return x, y

In [7]:
set_seed(42)

In [8]:
train_dataset = VideoDataset(vids_training, labels_training)
val_dataset   = VideoDataset(vids_val, labels_val)

In [9]:
g = torch.Generator().manual_seed(42)
perm = torch.randperm(len(train_dataset), generator=g).tolist()
train_sampler = SubsetRandomSampler(perm)

In [10]:
train_loader_seq = DataLoader(
    train_dataset, batch_size=32, shuffle=False, sampler=train_sampler,
    num_workers=0, collate_fn=collate_pad
)
val_loader_seq = DataLoader(
    val_dataset, batch_size=32, shuffle=False,
    num_workers=0, collate_fn=collate_pad
)

In [11]:
class TemporalBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1, dropout=0.1):
        super().__init__()
        pad = (kernel_size - 1) * dilation // 2  # 'same' padding aproximado
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size, padding=pad, dilation=dilation)
        self.norm = nn.BatchNorm1d(out_ch)
        self.drop = nn.Dropout(dropout)
        self.act  = nn.ReLU(inplace=True)
    def forward(self, x):  # x: (B, C, T)
        x = self.conv(x)
        x = self.norm(x)
        x = self.act(x)
        x = self.drop(x)
        return x

class MSTCN(nn.Module):
    """MS-TCN para clasificación de secuencias:
       Entrada:  (B, T, F)
       Conv1d:   (B, F, T)
       Capas:    'layers' bloques con dilaciones 1,2,4,...
       Salida:   (B, num_classes)"""
    def __init__(self, in_feat=258, hidden=128, layers=3, dropout=0.1, kernel_size=3, num_classes=4):
        super().__init__()
        stages = []
        in_ch = in_feat
        for i in range(layers):
            d = 2 ** i
            stages.append(TemporalBlock(in_ch, hidden, kernel_size=kernel_size, dilation=d, dropout=dropout))
            in_ch = hidden
        self.stages = nn.ModuleList(stages)
        self.head = nn.Linear(hidden, num_classes)

    def forward(self, x):          # x: (B, T, F)
        x = x.transpose(1, 2)      # -> (B, F, T)
        for blk in self.stages:
            x = blk(x)             # (B, hidden, T)
        x = x.mean(dim=-1)         # GAP en T -> (B, hidden)
        return self.head(x)        # (B, num_classes)

In [12]:
def optimize_ms_tcn(train_loader_seq, val_loader_seq, n_trials=10, max_epochs=50, save_prefix="hpo_ms_tcn"):
    set_seed(42)  # semilla global fija para el estudio

    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize", sampler=sampler,
                                pruner=optuna.pruners.MedianPruner())
    all_trials = []

    # Detectar automáticamente F (número de features)
    sample_x, _ = next(iter(train_loader_seq))
    in_feat = sample_x.shape[-1]

    def objective(trial):
        hidden  = trial.suggest_categorical("hidden", [96, 128, 160, 192])
        layers  = trial.suggest_int("layers", 2, 5)
        dropout = trial.suggest_float("dropout", 0.0, 0.5)
        ksz     = trial.suggest_categorical("kernel_size", [3, 5, 7])
        lr      = trial.suggest_float("lr", 5e-5, 5e-3, log=True)
        wd      = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)

        model = MSTCN(in_feat=in_feat, hidden=hidden, layers=layers,
                      dropout=dropout, kernel_size=ksz, num_classes=4).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

        best_val = 0.0
        for epoch in range(max_epochs):
            # --- Train ---
            model.train()
            for xb, yb in train_loader_seq:
                xb = xb.to(device); yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                optimizer.step()

            # --- Validation ---
            model.eval()
            y_true, y_pred = [], []
            with torch.no_grad():
                for xb, yb in val_loader_seq:
                    xb = xb.to(device)
                    preds = model(xb).argmax(dim=1)
                    y_true.extend(yb.numpy())
                    y_pred.extend(preds.cpu().numpy())
            val_acc = metrics_from_preds(y_true, y_pred)["accuracy"]

            if val_acc > best_val:
                best_val = val_acc

            trial.report(val_acc, epoch)
            if trial.should_prune():
                raise optuna.TrialPruned()

        all_trials.append({"trial": trial.number, "params": trial.params, "best_val_acc": best_val})
        return best_val

    study.optimize(objective, n_trials=n_trials)

    # Guardar resultados
    with open(f"{save_prefix}_all.json", "w") as f:
        json.dump(all_trials, f, indent=2)
    with open(f"{save_prefix}_best.json", "w") as f:
        json.dump({"best_params": study.best_params, "best_value": study.best_value}, f, indent=2)

    return study

In [13]:
if __name__ == "__main__":
    study_ms_tcn = optimize_ms_tcn(train_loader_seq, val_loader_seq, n_trials=40, max_epochs=50)
    print("Mejor resultado:", study_ms_tcn.best_value)
    print("Mejores hiperparámetros:", study_ms_tcn.best_params)

    # =============================================================================
    # [9] Reentrenar el mejor modelo de forma 100% reproducible
    # =============================================================================
    best = study_ms_tcn.best_params
    set_seed(42)  # entrenamiento único reproducible

    # Recuperar in_feat
    sample_x, _ = next(iter(train_loader_seq))
    in_feat = sample_x.shape[-1]

    model = MSTCN(
        in_feat=in_feat,
        hidden=best["hidden"],
        layers=best["layers"],
        dropout=best["dropout"],
        kernel_size=best["kernel_size"],
        num_classes=4
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=best["lr"], weight_decay=best["weight_decay"])

    for epoch in range(50):
        model.train()
        for xb, yb in train_loader_seq:
            xb = xb.to(device); yb = yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()

        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for xb, yb in val_loader_seq:
                xb = xb.to(device)
                preds = model(xb).argmax(dim=1)
                y_true.extend(yb.numpy())
                y_pred.extend(preds.cpu().numpy())
        val_acc = metrics_from_preds(y_true, y_pred)["accuracy"]
        print(f"Epoch {epoch+1:03d} | Val Acc: {val_acc:.4f}")

[I 2025-11-30 21:42:31,756] A new study created in memory with name: no-name-be42e895-5cc6-4704-8355-a32c47a06e73
[I 2025-11-30 21:43:19,279] Trial 0 finished with value: 0.9166666666666666 and parameters: {'hidden': 128, 'layers': 2, 'dropout': 0.07799726016810132, 'kernel_size': 5, 'lr': 0.0013035123791853842, 'weight_decay': 1.3934502251337584e-10}. Best is trial 0 with value: 0.9166666666666666.
[I 2025-11-30 21:43:44,407] Trial 1 finished with value: 0.9166666666666666 and parameters: {'hidden': 96, 'layers': 2, 'dropout': 0.15212112147976886, 'kernel_size': 3, 'lr': 0.0008369042894376068, 'weight_decay': 9.472334467618544e-10}. Best is trial 0 with value: 0.9166666666666666.
[I 2025-11-30 21:44:56,239] Trial 2 finished with value: 0.9111111111111111 and parameters: {'hidden': 192, 'layers': 2, 'dropout': 0.2571172192068058, 'kernel_size': 7, 'lr': 0.0001096524277832185, 'weight_decay': 2.853390105240223e-10}. Best is trial 0 with value: 0.9166666666666666.
[I 2025-11-30 21:45:41,

Mejor resultado: 0.9444444444444444
Mejores hiperparámetros: {'hidden': 96, 'layers': 2, 'dropout': 0.31953519251095136, 'kernel_size': 7, 'lr': 0.0002591144667122849, 'weight_decay': 1.9440339271153755e-07}
Epoch 001 | Val Acc: 0.7222
Epoch 002 | Val Acc: 0.7556
Epoch 003 | Val Acc: 0.8611
Epoch 004 | Val Acc: 0.5889
Epoch 005 | Val Acc: 0.7500
Epoch 006 | Val Acc: 0.9000
Epoch 007 | Val Acc: 0.8500
Epoch 008 | Val Acc: 0.7722
Epoch 009 | Val Acc: 0.8500
Epoch 010 | Val Acc: 0.8056
Epoch 011 | Val Acc: 0.8222
Epoch 012 | Val Acc: 0.6722
Epoch 013 | Val Acc: 0.7556
Epoch 014 | Val Acc: 0.8944
Epoch 015 | Val Acc: 0.7944
Epoch 016 | Val Acc: 0.7056
Epoch 017 | Val Acc: 0.6167
Epoch 018 | Val Acc: 0.8056
Epoch 019 | Val Acc: 0.8389
Epoch 020 | Val Acc: 0.7667
Epoch 021 | Val Acc: 0.8278
Epoch 022 | Val Acc: 0.6389
Epoch 023 | Val Acc: 0.8500
Epoch 024 | Val Acc: 0.8556
Epoch 025 | Val Acc: 0.8111
Epoch 026 | Val Acc: 0.9000
Epoch 027 | Val Acc: 0.7722
Epoch 028 | Val Acc: 0.6889
Epoch 02

In [14]:
import json
from metrics import metrics_from_preds
import torch
import torch.nn as nn
import torch.optim as optim

# --- Cargar los mejores hiperparámetros ---
print("Cargando los mejores hiperparámetros desde hpo_ms_tcn_best.json...")
try:
    with open("hpo_ms_tcn_best.json", "r") as f:
        best_hyperparams = json.load(f)["best_params"]
    print("Hiperparámetros cargados:")
    print(best_hyperparams)
except FileNotFoundError:
    print("Error: El archivo 'hpo_ms_tcn_best.json' no fue encontrado.")
    print("Por favor, ejecuta primero la celda de optimización para generar este archivo.")
    raise

# --- Configurar y entrenar el mejor modelo ---
set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Es necesario determinar el número de features de entrada desde los datos
try:
    sample_x, _ = next(iter(train_loader_seq))
    in_feat = sample_x.shape[-1]
except NameError:
    print("Error: 'train_loader_seq' no está definido. Asegúrate de haber ejecutado las celdas de preparación de datos.")
    raise

# Instanciar el modelo con los mejores hiperparámetros
best_model = MSTCN(
    in_feat=in_feat,
    hidden=best_hyperparams["hidden"],
    layers=best_hyperparams["layers"],
    dropout=best_hyperparams["dropout"],
    kernel_size=best_hyperparams["kernel_size"],
    num_classes=4
).to(device)

# Definir criterio de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(
    best_model.parameters(),
    lr=best_hyperparams["lr"],
    weight_decay=best_hyperparams["weight_decay"]
)

# Entrenamiento del modelo
epochs = 50  # Usamos las mismas épocas que en la búsqueda de hiperparámetros
print(f"\nEntrenando el mejor modelo MS-TCN durante {epochs} épocas...")

for epoch in range(epochs):
    best_model.train()
    total_loss = 0
    for xb, yb in train_loader_seq:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = best_model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader_seq):.4f}")

print("Entrenamiento completado.")

# --- Evaluación del modelo y cálculo de métricas ---
best_model.eval()
all_metrics = {}

def evaluate_seq_model(loader, model_name):
    y_true, y_pred = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            predictions = best_model(xb).argmax(dim=1)
            y_true.extend(yb.numpy())
            y_pred.extend(predictions.cpu().numpy())
    
    metrics = metrics_from_preds(y_true, y_pred)
    all_metrics[model_name] = metrics

print("\n--- Métricas del Modelo Final (MS-TCN) ---")
evaluate_seq_model(train_loader_seq, "Train")
evaluate_seq_model(val_loader_seq, "Validation")

for split_name, metrics in all_metrics.items():
    print(f"\nResultados en el conjunto de {split_name}:")
    for metric_name, value in metrics.items():
        if isinstance(value, float):
            print(f"  {metric_name}: {value:.4f}")
        else:
            print(f"  {metric_name}: {value}")

# --- Guardar el modelo entrenado ---
model_save_path = "ms_tcn_best_model.pth"
torch.save(best_model.state_dict(), model_save_path)
print(f"\nModelo guardado exitosamente en: {model_save_path}")


Cargando los mejores hiperparámetros desde hpo_ms_tcn_best.json...
Hiperparámetros cargados:
{'hidden': 96, 'layers': 2, 'dropout': 0.31953519251095136, 'kernel_size': 7, 'lr': 0.0002591144667122849, 'weight_decay': 1.9440339271153755e-07}

Entrenando el mejor modelo MS-TCN durante 50 épocas...
Epoch [10/50], Loss: 0.2867
Epoch [20/50], Loss: 0.1714
Epoch [30/50], Loss: 0.1318
Epoch [40/50], Loss: 0.0701
Epoch [50/50], Loss: 0.0911
Entrenamiento completado.

--- Métricas del Modelo Final (MS-TCN) ---

Resultados en el conjunto de Train:
  accuracy: 0.9461
  precision_macro: 0.9508
  recall_macro: 0.9461
  specificity_macro: 0.9820
  f1_macro: 0.9458
  balanced_accuracy: 0.9461
  confusion_matrix: [[252, 1, 0, 2], [0, 246, 0, 9], [2, 38, 214, 1], [1, 1, 0, 253]]

Resultados en el conjunto de Validation:
  accuracy: 0.9000
  precision_macro: 0.9220
  recall_macro: 0.9000
  specificity_macro: 0.9667
  f1_macro: 0.9020
  balanced_accuracy: 0.9000
  confusion_matrix: [[42, 0, 0, 3], [0, 34,